# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [169]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests # type: ignore
import json
from typing import List
from dotenv import load_dotenv # type: ignore
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [170]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [171]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
ed = Website("https://aquasec.com")
ed.links

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [172]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
print(link_system_prompt)

In [173]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [174]:
def enhance_brochure(user_prompt):
    user_prompt += '''\n\nPlease enhance the brochure with a brief description of the company, its products, and services. You Must Include sections {Mission},{Vision},{Values} and {Company_History} as seen in the Example below:\n
    
    Mission: To provide the best products and services to our customers
        ....
    Vision: To be the best company in the world
        ....
    Values: Integrity, Honesty, and Customer Satisfaction
        ....
    '''
    return user_prompt

In [ ]:
aqua = Website("https://aquasec.com")
print(enhance_brochure(get_links_user_prompt(aqua)))

In [ ]:
print(get_links_user_prompt(aqua))

In [175]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

In [ ]:
get_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [176]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://huggingface.co"))

In [222]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [178]:
def get_system_prompt():
    return system_prompt

In [220]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += enhance_brochure(user_prompt)
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt
    #print(user_prompt)

In [ ]:
get_brochure_user_prompt("Aqua Security", "https://aquasec.com")

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [158]:
def create_brochure(company_name, url, stream=False, lan='english'):
    if lan != 'english':
        system_prompt = f"Translate the brochure to {lan}."
    else:
        pass
        
    if stream:
        response = openai.ChatCompletion.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
              ],
            stream=True
        )
    else:
        response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    return result

In [86]:
def translate_to_hindi(text):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are an assistant that translates a brochure about a company into Hindi. Respond in markdown."},
            {"role": "user", "content": text}
          ],
    )
    return response.choices[0].message.content

In [180]:
def display_brochure(result):
    display(Markdown(result))

In [110]:
def create_english_brochure(company_name, url, stream):
    #company_name = input("Enter Company Name: ")
    #url = input("Enter Company URL: ")
    brochure = create_brochure(company_name, url, stream)
    return brochure

In [102]:
def create_hindi_brochure(company_name, url):
    #company_name = input("Enter Company Name: ")
    #url = input("Enter Company URL: ")
    brochure = translate_to_hindi(create_brochure(company_name, url))
    return brochure

In [ ]:
display_brochure(create_english_brochure())

In [ ]:
display_brochure(create_hindi_brochure())

In [ ]:
create_brochure("Anthropic", "https://anthropic.com")

In [ ]:
create_brochure("HuggingFace", "https://huggingface.com")

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [223]:
def generate_brochure(company_name, url, stream=True, lan='english'):
    system_prompt = get_system_prompt()

    if lan.lower() != 'english':
        system_prompt+= f" Output must be in markdown and language must be: {lan.upper()}."

    if stream:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
            ],
            stream=True
        )
        response_text = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in response:
            response_text += chunk.choices[0].delta.content or ''
            response_text = response_text.replace("```", "").replace("markdown", "")
            update_display(Markdown(response_text), display_id=display_handle.display_id)
    else:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
            ]
        )
        result = response.choices[0].message.content
        display(Markdown(result))

In [123]:
def generate_brochure_hindi_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": translate_to_hindi(get_brochure_user_prompt(company_name, url))}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [224]:
generate_brochure("Westpac", "https://westpac.com.au/", True, 'English')

Found links: {'links': [{'type': 'about page', 'url': 'https://westpac.com.au/about-westpac'}, {'type': 'careers page', 'url': 'https://westpac.com.au/about-westpac/careers'}, {'type': 'company page', 'url': 'https://westpac.com.au/about-westpac/westpac-group'}, {'type': 'investor center', 'url': 'https://westpac.com.au/about-westpac/investor-centre'}, {'type': 'sustainability page', 'url': 'https://westpac.com.au/about-westpac/sustainability'}, {'type': 'indigenous page', 'url': 'https://westpac.com.au/about-westpac/indigenous'}]}



# Welcome to Westpac! 🏦

Where a skip, hop, and a click leads you to a whole new world of banking fun! Whether you're a personal saver, a business mogul, or a corporate tycoon, Westpac has everything you need – well, maybe not a pet dinosaur, but we have a great range of financial services!

---

## Our Mission 🎯
To provide the best products and services to our customers, making banking so easy you might just want to start a dance party!

## Our Vision 🌍
To be the best company in the world, or at least in the top three if we’re being realistic. Right behind pizza delivery and superhero vendors.

## Our Values 🏅
- **Integrity**: We mean what we say. No funny business here, except for the kind that involves fancy coffee machines in the office!
- **Customer Satisfaction**: If our customers aren’t happy, we’re like a puppy without a bone – very, very sad.
- **Innovation**: We’re not just banking; we’re trying to engineer the future of finance—without the flux capacitor (but we're open to ideas!).

---

## Company History 📅
Founded in the 1817s, Westpac is older than a good bottle of wine and almost as wise! Originally a bank for the people, we’ve evolved and now offer everything from home loans and credit cards to margin lending for those who like to live on the edge (financially, of course).

---

## Our Products and Services 📦
Whether you need a bank account, a loan, or just a friendly chat about your financial future, we've got you covered! Here's just a snippet of what we offer:

- **Personal Banking**: Think bank accounts, home loans, credit cards, and maybe a secret stash for rainy days.
- **Business Banking**: We're all about helping your enterprise grow – no, we don’t mean a budding investment in spray tans for cats... unless that’s your thing!
- **Corporate Banking**: Tailored for big fish in the finance pond – think corporate finance, trade & supply chain finance, and maybe a golden parachute or two.

---

## Company Culture 🏢
At Westpac, we believe in a workplace that’s as open as our bank accounts! We embrace diversity, inclusivity, and plenty of chances to get together for team-building karaoke nights (we’ll bring the pitch pipes). 

We also continually promote sustainability and community support, because saving the planet while crunching numbers is the best kind of multitasking! 🌱

---

## Join Our Team! 🙌
Looking for a career with growth potential and a dash of fun? Whether you’re a number-crunching wizard or a customer service superstar, we’re excited to have you aboard! Check out our careers page—suit up, and let’s make banking fun together!

---

**Westpac: Banking with a Smile (and maybe a little bit of a giggle)!** 

For more information or to start your financial adventure with us, [visit our website](#)!



In [ ]:
stream_hindi_brochure("Anz Bank", "https://anz.com.au/")

In [ ]:
stream_brochure("Woodleigh Waters Dental Surgery", "https://woodleighwatersdentalsurgery.com.au/")

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>